In [4]:
%load_ext autoreload

In [5]:
import os
print( os.getcwd() )
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )

/var/genie-in-the-box/src/notebooks
/var/genie-in-the-box/src


In [6]:
%autoreload

import sys
import os
import json


# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" )

print( sys.path )


import lib.util             as du
path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import lib.util_pandas      as up
import lib.util_stopwatch   as sw
import lib.util_code_runner as ucr

from solution_snapshot_mgr import SolutionSnapshotManager
from agent_refactoring     import RefactoringAgent

import openai

debug   = True
verbose = True


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var/genie-in-the-box/src/notebooks', '/usr/local/lib/python310.zip', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/site-packages', '/var/genie-in-the-box/src/lib']
Added [/var/genie-in-the-box/src] to sys.path
pwd [/var/genie-in-the-box/src]

/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug
/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev
/usr/local/lib/python3.10
/usr/local/lib/python3.10/lib-dynload
/usr/local/lib/python3.10/site-packages
/usr/local/lib/python310.zip
/var/genie-in-the-box/src
/var/genie-in-the-box/src/lib
/var/genie-in-the-box/src/notebooks


In [77]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def ask_chat_gpt_text( preamble, query, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are chatGPT, a helpful chatbot"
question = "What's your name?"

ask_chat_gpt_text( preamble, question, model=GPT_4 )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,656 ms


"My name is ChatGPT. I'm an artificial intelligence designed to assist with information and conversation."

In [167]:
import pandas as pd
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df[ ( df.event_type == 'Concert' ) ].start_date.value_counts()


start_date
2023-11-01    2
2023-10-28    2
2023-09-30    2
2023-10-13    1
2023-10-30    1
2023-10-22    1
2023-10-21    1
2023-10-20    1
2023-10-19    1
2023-10-15    1
2023-09-01    1
2023-09-06    1
2023-10-03    1
2023-09-28    1
2023-09-27    1
2023-09-20    1
2023-09-12    1
2023-09-09    1
2023-10-07    1
Name: count, dtype: int64

In [168]:
df.event_type.value_counts()

event_type
Conference      24
Concert         22
Performance     21
Appointment     17
Anniversary     17
Workout         17
Birthday        16
Workshop        15
Subscription    14
Meeting         12
Interview       11
TODO             7
Todo             7
Name: count, dtype: int64

In [223]:
%autoreload
    
def begin_refactoring_similar_questions( question ):
    
    path_to_snapshots = du.get_project_root() + "/src/conf/long-term-memory/solutions/"
    snapshot_mgr      = SolutionSnapshotManager( path_to_snapshots, debug=False )
    exemplar_snapshot = snapshot_mgr.get_snapshots_by_question( question )[ 0 ][ 1 ]
    similar_snapshots = snapshot_mgr.get_snapshots_by_code_similarity( exemplar_snapshot, threshold=90.0 )
        
    agent = RefactoringAgent( 
        similar_snapshots=similar_snapshots, 
        path_to_solutions="/src/conf/long-term-memory/solutions", 
        debug=True, verbose=True 
    )
    
    if not agent.is_promptable():
        print( f"No similar snapshots found for [{du.truncate_string( exemplar_snapshot.question, max_len=32 )}]" )
    else:
        agent.run_prompt()
        
    return agent
    
agent = begin_refactoring_similar_questions( "What concerts do I have this week?" )    
    


------------------------------------------------------------------------------------------------------------------------
- Found [36] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapshot.question [what day is today]
Synonymous question [whats todays day and date] for snapshot.question [what day is today]
Synonymous question [what is the day and date for today] for snapshot.question [what day is today]
Synonymous question [whats the date] for snapshot.question [what day is today]
Synonymous question [say what day is today] for snapshot.question [what day is today]
Synonymous question [hey what day is today] for snapshot.question [what day is today]
Synonymous question [whats

In [224]:
# Print dictionary to console, formatted with indentation, etc.
print( json.dumps( agent.response_dict[ "gpt_function_signatures" ], indent=4 ).replace( "'", '"' ).strip( '"' ) )


[{        "name": "get_this_weeks_events_by_type",        "description": "Gets the events of a certain type that occur in the current week",        "parameters": {            "type": "object",            "properties": {                "df": {                    "type": "object",                    "description": "The dataframe containing the events",                },                "event_type": {                    "type": "string",                    "description": "The type of the event",                },            },            "required": ["df", "event_type"],        },    }]


In [141]:
%autoreload
agent.refactor_code( debug=True )

Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.py]... Done!
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.json]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_0.py]... Done!

File     count: [0]
File file_name: [util_calendaring_0.py]
File repo_path: [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.json]
File   io_path: [/var/genie-in-the-box/io/lib/autogen/util_calendaring_0.py]
File    abbrev: [uc0]
File    import: [import lib.autogen.util_calendaring_0 as uc0]
File call_template: [uc0.get_this_weeks_events_by_type( df, 'event_type' )]
File     signature: [{'name': 'get_this_weeks_events_by_type', 'description': 'Gets the events of a specific type for the current week', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['

In [81]:
%autoreload
agent.run_code()

last command, before [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))]:
return_type [string]
ERROR: return_type [string] and last command [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))] are incompatible!
last command,  after [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_0 as uc0
print(uc0.get_this_weeks_events_by_type(df, 'Concert'))

Executing /var/genie-in-the-box/io/code.py... Done!

------------------------------------------------------------------------------------------------------------------------
- Code output
------------------------------------------------------------------------------------------------------------------------

{"start_date":1697155200000,"end_date":1697155200000,"start_time":"00:

[{'return_code': 0,
  'output': '{"start_date":1697155200000,"end_date":1697155200000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"highest","name":"Leroy Ruiz","relationship":"father","description_who_what_where":"Concert of Leroy Ruiz at the city center"}\n{"start_date":1697328000000,"end_date":1697328000000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"high","name":"Tom Ruiz","relationship":"brother","description_who_what_where":"Concert of Tom Ruiz at the city center"}'},
 {'return_code': 0,
  'output': '{"start_date":1696982400000,"end_date":1696982400000,"start_time":"00:00","end_time":"23:59","event_type":"Birthday","recurrent":true,"recurrence_interval":"5 month","priority_level":"highest","name":"Sue","relationship":"sister","description_who_what_where":"Sue\'s birthday party at their favorite bar"}\n{"start_date":16969

### Prototype new function prompt

In [148]:
%autoreload
signature_paths = du.find_files_with_prefix_and_suffix( du.get_project_root() + "/src/lib/autogen/", "util_calendaring", ".json" )
# Given a list of file paths, load each file into a string and append to a list

signatures = du.get_files_as_strings( signature_paths )
signatures = [ json.loads( s ) for s in signatures ]
du.print_list( signatures )

signature_dict = {}
for signature in signatures:
    signature_dict[ signature[ "name" ] ] = signature
    
signature_dict

{'name': 'get_this_weeks_events_by_type', 'description': 'Gets the events of a specific type for the current week', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['df', 'event_type']}, 'call_template': "uc0.get_this_weeks_events_by_type( df, 'event_type' )", 'import_as': 'import lib.autogen.util_calendaring_0 as uc0'}


{'get_this_weeks_events_by_type': {'name': 'get_this_weeks_events_by_type',
  'description': 'Gets the events of a specific type for the current week',
  'parameters': {'type': 'object',
   'properties': {'df': {'type': 'object',
     'description': 'The DataFrame containing the events data'},
    'event_type': {'type': 'string',
     'description': 'The type of event to filter by'}},
   'required': ['df', 'event_type']},
  'call_template': "uc0.get_this_weeks_events_by_type( df, 'event_type' )",
  'import_as': 'import lib.autogen.util_calendaring_0 as uc0'}}

In [235]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

# example: https://www.mlq.ai/gpt-function-calling-getting-started/#:~:text=Step%201%3A%20Defining%20functions%20for,available%20functions%20it%20can%20call
def get_function_from_gpt( preamble, query, functions, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        functions=functions,
        function_call="auto",
        temperature=0,
        max_tokens=2000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    message = response[ "choices" ][ 0 ][ "message" ]
    
    results_dict = {
        "is_function_call": False,
                 # "message": message,
           "function_name": None,   
           "function_args": None,
                 "content": None
    }
    
    if message.get( "function_call" ):
        results_dict[ "is_function_call" ] = True
        results_dict[ "function_name"    ] = message[ "function_call" ][ "name" ]
        results_dict[ "function_args"    ] = json.loads( message[ "function_call" ][ "arguments" ] )
        # title case the event_type value
        results_dict[ "function_args"    ][ "event_type"]  = results_dict[ "function_args" ][ "event_type"].title()
    else:
        results_dict[ "content" ] = response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()
    
    return results_dict

preamble = "You are designed to take a question and map it to a function."
# question = "Do I have any appointments this week?"
question = "What appointments Do I have this week?"

gpt_function_signature_results = get_function_from_gpt( preamble, question, list( signature_dict.values() ), model=GPT_4, debug=False )


Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 2,386 ms


In [236]:
gpt_function_signature_results

{'is_function_call': True,
 'function_name': 'get_this_weeks_events_by_type',
 'function_args': {'event_type': 'Appointment'},
 'content': None}

In [208]:
import_as     = signature_dict[ gpt_function_signature_results[ "function_name" ] ][ "import_as" ]
call_template = signature_dict[ gpt_function_signature_results[ "function_name" ] ][ "call_template" ]
call_template

"uc0.get_this_weeks_events_by_type( df, 'event_type' )"

In [209]:
# For now this only replaces one argument name with a value, in the future it may handle multiple non-'df' arguments
for arg, value in gpt_function_signature_results[ "function_args" ].items():
    # print( arg, value )
    call_template = call_template.replace( arg, f"{value}" )
    print( call_template )

uc0.get_this_weeks_events_by_type( df, 'Appointment' )


In [210]:
code = [
    import_as,
    call_template
]
du.print_list( code )

import lib.autogen.util_calendaring_0 as uc0
uc0.get_this_weeks_events_by_type( df, 'Appointment' )


In [211]:
%autoreload

def get_function_signatures( debug=False ):
    
    signature_paths = du.find_files_with_prefix_and_suffix( du.get_project_root() + "/src/lib/autogen/", "util_calendaring", ".json" )
    # Given a list of file paths, load each file into a string and append to a list
    
    signatures = du.get_files_as_strings( signature_paths )
    signatures = [ json.loads( s ) for s in signatures ]
    if debug: du.print_list( signatures )
    
    signature_dict = {}
    for signature in signatures:
        signature_dict[ signature[ "name" ] ] = signature
        
    return signature_dict

signature_dict = get_function_signatures()
signature_dict

{'get_this_weeks_events_by_type': {'name': 'get_this_weeks_events_by_type',
  'description': 'Gets the events of a specific type for the current week',
  'parameters': {'type': 'object',
   'properties': {'df': {'type': 'object',
     'description': 'The DataFrame containing the events data'},
    'event_type': {'type': 'string',
     'description': 'The type of event to filter by'}},
   'required': ['df', 'event_type']},
  'call_template': "uc0.get_this_weeks_events_by_type( df, 'event_type' )",
  'import_as': 'import lib.autogen.util_calendaring_0 as uc0'}}

In [ ]:
question = "what appointments do i have today"

In [237]:
def get_function_signatures( debug=False ):
    
    signature_paths = du.find_files_with_prefix_and_suffix( du.get_project_root() + "/src/lib/autogen/", "util_calendaring", ".json" )
    # Given a list of file paths, load each file into a string and append to a list
    
    signatures = du.get_files_as_strings( signature_paths )
    signatures = [ json.loads( s ) for s in signatures ]
    if debug: du.print_list( signatures )
    
    signature_dict = {}
    for signature in signatures:
        signature_dict[ signature[ "name" ] ] = signature
        
    return signature_dict
    
def get_code_for_question( question, debug=False ):
    
    signature_dict                 = get_function_signatures()
    gpt_function_signature_results = get_function_from_gpt( preamble, question, list( signature_dict.values() ), model=GPT_4, debug=debug )
    
    # If the response is not a function call, return an empty list
    if not gpt_function_signature_results[ "is_function_call" ]:
        
        du.print_banner( f"Not a function call [{question}]" )
        print( gpt_function_signature_results[ "content" ] )
        
        return []
    else:
        import_as     = signature_dict[ gpt_function_signature_results[ "function_name" ] ][ "import_as" ]
        call_template = signature_dict[ gpt_function_signature_results[ "function_name" ] ][ "call_template" ]
        
        # For now this only replaces one argument name with a value, in the future it may handle multiple non-'df' arguments
        for arg, value in gpt_function_signature_results[ "function_args" ].items():
            # print( arg, value )
            call_template = call_template.replace( arg, f"{value}" )
        
        code = [
            import_as,
            call_template
        ]
        # du.print_list( code )
        
        return code
    
# code = get_code_for_question( "What appointments do I have this week?" )
# code = get_code_for_question( "What todos do I have this week?" )
code = get_code_for_question( "What should I have for dinner tonight?" )
code

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,052 ms

------------------------------------------------------------------------------------------------------------------------
- Not a function call [What should I have for dinner tonight?]
------------------------------------------------------------------------------------------------------------------------

Sorry, I can't assist with that.


[]

In [230]:
code = get_code_for_question( "What Happy hours do I have this week?" )
code

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,923 ms

------------------------------------------------------------------------------------------------------------------------
- Not a function call [What Happy hours do I have this week?]
------------------------------------------------------------------------------------------------------------------------

{
  "function": "functions.get_this_weeks_events_by_type",
  "arguments": {
    "event_type": "Happy hours"
  }
}


[]

In [238]:
code = get_code_for_question( "Do I have any appointments for tomorrow?" )
code

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,335 ms

------------------------------------------------------------------------------------------------------------------------
- Not a function call [Do I have any appointments for tomorrow?]
------------------------------------------------------------------------------------------------------------------------

I'm sorry, but I can only provide information about events for the current week or today.


[]

In [239]:
code = get_code_for_question( "Do I have any appointments for today?", debug=True )
code

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 2,427 ms

{
    "id": "chatcmpl-87uadFBSwYqfpDoooWSPSdXq0Gtsx",
    "object": "chat.completion",
    "created": 1696896519,
    "model": "gpt-4-0613",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": null,
                "function_call": {
                    "name": "get_todays_events_by_type",
                    "arguments": "{\n  \"event_type\": \"appointment\"\n}"
                }
            },
            "finish_reason": "function_call"
        }
    ],
    "usage": {
        "prompt_tokens": 131,
        "completion_tokens": 21,
        "total_tokens": 152
    }
}


['import lib.autogen.util_calendaring_1 as uc1',
 "uc1.get_todays_events_by_type( df, 'Appointment' )"]

In [240]:
# execute code
import lib.util_code_runner as ucr
response = ucr.assemble_and_run_solution( code, path="/src/conf/long-term-memory/events.csv" )
response

last command, before [uc1.get_todays_events_by_type( df, 'Appointment' )]:
return_type [string]
ERROR: return_type [string] and last command [uc1.get_todays_events_by_type( df, 'Appointment' )] are incompatible!
last command,  after [uc1.get_todays_events_by_type( df, 'Appointment' )]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_1 as uc1
uc1.get_todays_events_by_type( df, 'Appointment' )

Executing /var/genie-in-the-box/io/code.py... Done!


{'return_code': 0, 'output': 'No results returned'}

In [182]:
code = get_code_for_question( "What todos do I have this week?" )
code

{'name': 'get_this_weeks_events_by_type', 'description': 'Gets the events of a specific type for the current week', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['df', 'event_type']}, 'call_template': "uc0.get_this_weeks_events_by_type( df, 'event_type' )", 'import_as': 'import lib.autogen.util_calendaring_0 as uc0'}
Asking ChatGPT [gpt-3.5-turbo-0613]...
Asking ChatGPT [gpt-3.5-turbo-0613]... in 1,160 ms


['import lib.autogen.util_calendaring_0 as uc0',
 "uc0.get_this_weeks_events_by_type( df, 'Todo' )"]

In [186]:
code = get_code_for_question( "What Should I watch on TV tonight?" )
code

{'name': 'get_this_weeks_events_by_type', 'description': 'Gets the events of a specific type for the current week', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['df', 'event_type']}, 'call_template': "uc0.get_this_weeks_events_by_type( df, 'event_type' )", 'import_as': 'import lib.autogen.util_calendaring_0 as uc0'}
Asking ChatGPT [gpt-3.5-turbo-0613]...
Asking ChatGPT [gpt-3.5-turbo-0613]... in 619 ms


['import lib.autogen.util_calendaring_0 as uc0',
 "uc0.get_this_weeks_events_by_type( df, 'Tv' )"]

In [195]:
code = get_code_for_question( "What Should I eat tonight?" )
code

Asking ChatGPT [gpt-3.5-turbo-0613]...
Asking ChatGPT [gpt-3.5-turbo-0613]... in 1,378 ms


['import lib.autogen.util_calendaring_0 as uc0',
 "uc0.get_this_weeks_events_by_type( df, 'Dinner' )"]

In [183]:
import lib.util_code_runner as ucr

code_response = ucr.assemble_and_run_solution( code, path="/src/conf/long-term-memory/events.csv" )
code_response

last command, before [uc0.get_this_weeks_events_by_type( df, 'Todo' )]:
return_type [string]
ERROR: return_type [string] and last command [uc0.get_this_weeks_events_by_type( df, 'Todo' )] are incompatible!
last command,  after [uc0.get_this_weeks_events_by_type( df, 'Todo' )]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_0 as uc0
uc0.get_this_weeks_events_by_type( df, 'Todo' )

Executing /var/genie-in-the-box/io/code.py... Done!


{'return_code': 0, 'output': 'No results returned'}

In [225]:
agent = begin_refactoring_similar_questions( "What appointments do I have today?" )
agent.


------------------------------------------------------------------------------------------------------------------------
- Found [36] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapshot.question [what day is today]
Synonymous question [whats todays day and date] for snapshot.question [what day is today]
Synonymous question [what is the day and date for today] for snapshot.question [what day is today]
Synonymous question [whats the date] for snapshot.question [what day is today]
Synonymous question [say what day is today] for snapshot.question [what day is today]
Synonymous question [hey what day is today] for snapshot.question [what day is today]
Synonymous question [whats

In [244]:
agent.response_dict

{'thoughts': 'The three code snippets are very similar, they all filter a DataFrame based on the current date and a specific event type. The only difference between them is the event type. Therefore, we can create a single function that takes the DataFrame and the event type as parameters, and returns the filtered DataFrame. If the query does not return a result, the function will return an empty DataFrame.',
 'code': ['import pandas as pd',
  'def get_todays_events_by_type(df, event_type):',
  "    today = pd.Timestamp('now').normalize()",
  "    events_today = df[(df['event_type'] == event_type) & (df['start_date'] <= today) & (df['end_date'] >= today)]",
  '    return events_today'],
 'function_name': 'get_todays_events_by_type',
 'parameters': 'df, event_type',
 'gpt_function_signatures': "[ { 'name': 'get_todays_events_by_type', 'description': 'Gets the events of a specific type that are happening today', 'parameters': { 'type': 'object', 'properties': { 'df': { 'type': 'object', 

In [226]:
agent.refactor_code( debug=True )

Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_1.py]... Done!
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_1.json]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_1.py]... Done!

File     count: [1]
File file_name: [util_calendaring_1.py]
File repo_path: [/var/genie-in-the-box/src/lib/autogen/util_calendaring_1.json]
File   io_path: [/var/genie-in-the-box/io/lib/autogen/util_calendaring_1.py]
File    abbrev: [uc1]
File    import: [import lib.autogen.util_calendaring_1 as uc1]
File call_template: [uc1.get_todays_events_by_type( df, 'event_type' )]
File     signature: [{'name': 'get_todays_events_by_type', 'description': 'Gets the events of a specific type that are happening today', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['df',

In [227]:
agent.run_code()

last command, before [print(uc1.get_todays_events_by_type(df, 'Appointment').to_json(orient='records', lines=True))]:
return_type [string]
ERROR: return_type [string] and last command [print(uc1.get_todays_events_by_type(df, 'Appointment').to_json(orient='records', lines=True))] are incompatible!
last command,  after [print(uc1.get_todays_events_by_type(df, 'Appointment').to_json(orient='records', lines=True))]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_1 as uc1
print(uc1.get_todays_events_by_type(df, 'Appointment').to_json(orient='records', lines=True))

Executing /var/genie-in-the-box/io/code.py... Done!

------------------------------------------------------------------------------------------------------------------------
- Code output
----------------------------------------

[{'return_code': 0,
  'output': '{"start_date":1696896000000,"end_date":1696982400000,"start_time":"16:22","end_time":"07:17","event_type":"Appointment","recurrent":false,"recurrence_interval":null,"priority_level":"low","name":"Barbara Jane Ruiz","relationship":"mother","description_who_what_where":"Appointment with Barbara Jane Ruiz at the clinic"}'},
 {'return_code': 0,
  'output': '{"start_date":1696636800000,"end_date":1697241600000,"start_time":"00:00","end_time":"23:59","event_type":"Conference","recurrent":false,"recurrence_interval":null,"priority_level":"low","name":"Bob","relationship":"brother","description_who_what_where":"Conference with Bob on AI advancements"}'},
 {'return_code': 0, 'output': 'No results returned'}]

In [242]:
def get_and_run_code_for_question( question ):
    
    code = get_code_for_question( question )
    
    if code:
        code_response = ucr.assemble_and_run_solution( code, path="/src/conf/long-term-memory/events.csv" )
        return code_response
    else:
        return None
    
code_response = get_and_run_code_for_question( "Do I have any appointments today?" )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 4,573 ms

last command, before [uc1.get_todays_events_by_type( df, 'Appointment' )]:
return_type [string]
ERROR: return_type [string] and last command [uc1.get_todays_events_by_type( df, 'Appointment' )] are incompatible!
last command,  after [uc1.get_todays_events_by_type( df, 'Appointment' )]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_1 as uc1
uc1.get_todays_events_by_type( df, 'Appointment' )

Executing /var/genie-in-the-box/io/code.py... Done!


In [243]:
code_response

{'return_code': 0, 'output': 'No results returned'}